### Check each Ameri site csv file, the last year will missing some rows

In [23]:
import os
import pandas as pd
import numpy as np
import glob
import pickle

import calendar

from zipfile import ZipFile

In [2]:
zip_dir = './ameri_data/'
all_zipfiles = glob.glob(zip_dir + "*.zip")

In [3]:
len(all_zipfiles)

333

In [4]:
# Get those name from the c file "one_flux\ONEFlux\oneflux_steps\qc_auto\src\dataset.c"  const char *const var_names[]
var_names = ["CO2","H2O","ZL","FC","FC_SSITC_TEST","H","H_SSITC_TEST","LE","LE_SSITC_TEST","USTAR",
"TR","SB","SC","SLE","SH","P","SW_OUT","SW_IN","NETRAD","SW_DIF","PPFD_IN","APAR","TA","PA","T_CANOPY","T_BOLE","TS","SWC","G",
"RH","WD","WS","TAU","LW_IN","NEE","VPD","itpVPD","itpSW_IN","itpPPFD_IN","itpTA","itpTS","itpSWC","itpP","itpRH","FETCH_FILTER",
]

In [5]:
must_have = ['TA','RH', 'PA', 'WS','P', 'SW_IN', 'LW_IN']

In [6]:
#split rows per years
# return year list and start and end position for each year
def split_records(src_data: pd.DataFrame) -> (list, list, list):
    years = []
    start_pos = []
    end_pos = []

    start_year = ''
    rows = src_data['TIMESTAMP_END'].count()
    for idx in range(rows):
        if start_year == '':
            start_year = src_data.iloc[idx]['TIMESTAMP_START'][:4]
            years.append(start_year)
            start_pos.append(idx)
        else:
            if start_year == src_data.iloc[idx]['TIMESTAMP_START'][:4]:
                _end = idx
            else:
                end_pos.append(_end)
                start_year = src_data.iloc[idx]['TIMESTAMP_START'][:4]
                years.append(start_year)
                start_pos.append(idx)

    end_pos.append(_end)
    
    return years, start_pos, end_pos

In [7]:
for era_site_list in os.walk('./output/'):
    break

era_site_list[1]

['US-xWR',
 'US-Syv',
 'US-CC2',
 'US-MC1',
 'US-Kon',
 'US-Dk3',
 'US-Bo2',
 'US-xJE',
 'US-CRT',
 'US-TrS',
 'US-xSJ',
 'US-Wi1',
 'US-DFK',
 'US-SRG',
 'US-Wlr',
 'US-DPW',
 'US-PFf',
 'CA-TP3',
 'US-MOz',
 'US-UiA',
 'US-Wi8',
 'US-GLE',
 'US-PSH',
 'US-PFh',
 'US-MtB',
 'US-CF2',
 'US-xST',
 'US-MVF',
 'US-xSL',
 'US-Jo1',
 'US-Ho3',
 'US-Mi3',
 'US-MMS',
 'CA-TP2',
 'US-Lin',
 'US-Wi3',
 'US-xSR',
 'US-Ne1',
 'US-Dia',
 'US-PHM',
 'US-PFa',
 'US-Ha2',
 'US-xDL',
 'US-RGo',
 'US-Pnp',
 'US-JRn',
 'US-Ho2',
 'US-Bi1',
 'CA-MA1',
 'US-Fuf',
 'US-MVW',
 'US-KFS',
 'US-Oho',
 'US-Mpj',
 'US-SP2',
 'US-Blo',
 'CA-DB2',
 'US-AR1',
 'US-Akn',
 'US-xJR',
 'US-PFd',
 'US-NR1',
 'US-CS5',
 'US-Vcp',
 'US-Rms',
 'US-Rwf',
 'US-xSP',
 'CA-MA3',
 'US-xUK',
 'CA-ARB',
 'US-TrB',
 'US-xUN',
 'US-xRM',
 'US-PFc',
 'MX-Aog',
 'US-Mj1',
 'US-CF4',
 'US-xBL',
 'US-DS3',
 'US-Vcm',
 'US-NC4',
 'US-Vcs',
 'US-Ro5',
 'US-Wi9',
 'US-Wrc',
 'US-CS3',
 'US-RGA',
 'US-xWD',
 'US-Tw3',
 'US-xSC',
 'US-ASM',

In [8]:
len(era_site_list[1])

283

In [35]:
records = []
for zip_file in all_zipfiles:
    #if zip_file in processed_list:
    #    continue
        
    #if zip_file in bad_list:
    #    continue
    
    
    file_name = os.path.basename(zip_file)

    
    file_name = file_name.split('.')[0]
    site_name = file_name.split('_')[1]
    
    #don't create qcv files if no era file existing
    if site_name not in era_site_list[1]:
        continue
    
    #site_info = pd_site[pd_site['site_id'] == site_name]

    with ZipFile(zip_file, 'r') as zip_folder:
        file_list = zip_folder.namelist()
        if len(file_list) > 1:
            for _file in file_list:
                name_split = _file.split('.')

                if name_split[1] == 'csv':
                    df = pd.read_csv(zip_folder.open(_file),dtype={'TIMESTAMP_START':str, 'TIMESTAMP_END':str},skiprows=2)
                    if df.iloc[0]['TIMESTAMP_END'][-4:] == '0030':
                        record_interval = 'halfhourly'
                    else:
                        record_interval = 'hourly'
                        
                    print('Process zip file {}........'.format(zip_file))

                    years, start_pos, end_pos = split_records(df)

                    for year, start, end in zip(years, start_pos, end_pos):
                        if int(year) == 2021:
                            break

                    last_year = year
                    if calendar.isleap(int(last_year)):
                        if record_interval == 'halfhourly':
                            total_records = 17568
                        else:
                            total_records = 8784
                    else:
                        if record_interval == 'halfhourly':
                            total_records = 17520
                        else:
                            total_records = 8760
                    
                    last_year_data = df.iloc[start:end+1,:]
                    records_num = last_year_data['TIMESTAMP_START'].count()
                    if records_num != total_records:
                        _row = dict()
                        _row['file_name'] = _file
                        _row['year'] = last_year
                        _row['existing_rows'] = records_num
                        _row['add_rows'] = total_records - records_num
                        
                        records.append(_row)
                        print("{} {} exist {}, need add {}".format(_file, last_year, records_num, total_records - records_num))
                        break


Process zip file ./ameri_data/AMF_US-xGR_BASE-BADM_6-5.zip........
Process zip file ./ameri_data/AMF_US-xAE_BASE-BADM_5-5.zip........
Process zip file ./ameri_data/AMF_US-PFs_BASE-BADM_1-5.zip........
AMF_US-PFs_BASE_HH_1-5.csv 2019 exist 14592, need add 2928
Process zip file ./ameri_data/AMF_US-ARM_BASE-BADM_11-5.zip........
AMF_US-ARM_BASE_HH_11-5.csv 2021 exist 8484, need add 9036
Process zip file ./ameri_data/AMF_US-xNW_BASE-BADM_5-5.zip........
Process zip file ./ameri_data/AMF_US-xJR_BASE-BADM_6-5.zip........
Process zip file ./ameri_data/AMF_US-JRn_BASE-BADM_1-5.zip........
AMF_US-JRn_BASE_HH_1-5.csv 2019 exist 4320, need add 13200
Process zip file ./ameri_data/AMF_US-KM1_BASE-BADM_4-5.zip........
Process zip file ./ameri_data/AMF_US-KUT_BASE-BADM_1-1.zip........
Process zip file ./ameri_data/AMF_US-xBL_BASE-BADM_5-5.zip........
Process zip file ./ameri_data/AMF_US-Tw5_BASE-BADM_3-5.zip........
AMF_US-Tw5_BASE_HH_3-5.csv 2020 exist 1392, need add 16176
Process zip file ./ameri_d

Process zip file ./ameri_data/AMF_US-PFn_BASE-BADM_1-5.zip........
AMF_US-PFn_BASE_HH_1-5.csv 2019 exist 14592, need add 2928
Process zip file ./ameri_data/AMF_US-KS1_BASE-BADM_3-5.zip........
Process zip file ./ameri_data/AMF_US-NC3_BASE-BADM_4-5.zip........
Process zip file ./ameri_data/AMF_US-CS3_BASE-BADM_3-5.zip........
AMF_US-CS3_BASE_HH_3-5.csv 2020 exist 6048, need add 11520
Process zip file ./ameri_data/AMF_US-xSB_BASE-BADM_5-5.zip........
Process zip file ./ameri_data/AMF_US-A32_BASE-BADM_1-5.zip........
Process zip file ./ameri_data/AMF_US-Mi2_BASE-BADM_1-5.zip........
AMF_US-Mi2_BASE_HH_1-5.csv 2017 exist 5722, need add 11798
Process zip file ./ameri_data/AMF_US-RGA_BASE-BADM_1-5.zip........
Process zip file ./ameri_data/AMF_CA-ER1_BASE-BADM_3-5.zip........
AMF_CA-ER1_BASE_HH_3-5.csv 2021 exist 5759, need add 11761
Process zip file ./ameri_data/AMF_US-PFh_BASE-BADM_1-5.zip........
AMF_US-PFh_BASE_HH_1-5.csv 2019 exist 14592, need add 2928
Process zip file ./ameri_data/AMF_U

Process zip file ./ameri_data/AMF_US-Bo2_BASE-BADM_2-1.zip........
Process zip file ./ameri_data/AMF_US-Blo_BASE-BADM_4-5.zip........
Process zip file ./ameri_data/AMF_US-Dk2_BASE-BADM_4-5.zip........
Process zip file ./ameri_data/AMF_US-Wi3_BASE-BADM_3-5.zip........
Process zip file ./ameri_data/AMF_CA-ARB_BASE-BADM_1-5.zip........
AMF_CA-ARB_BASE_HH_1-5.csv 2015 exist 17472, need add 48
Process zip file ./ameri_data/AMF_US-Mj2_BASE-BADM_1-5.zip........
AMF_US-Mj2_BASE_HH_1-5.csv 2014 exist 17519, need add 1
Process zip file ./ameri_data/AMF_US-MVW_BASE-BADM_1-5.zip........
AMF_US-MVW_BASE_HH_1-5.csv 2017 exist 12987, need add 4533
Process zip file ./ameri_data/AMF_US-SP4_BASE-BADM_3-5.zip........
Process zip file ./ameri_data/AMF_US-Myb_BASE-BADM_13-5.zip........
Process zip file ./ameri_data/AMF_US-MMS_BASE-BADM_21-5.zip........
Process zip file ./ameri_data/AMF_US-UM3_BASE-BADM_1-5.zip........
AMF_US-UM3_BASE_HH_1-5.csv 2014 exist 11424, need add 6096
Process zip file ./ameri_data/

Process zip file ./ameri_data/AMF_US-KL1_BASE-BADM_3-5.zip........
Process zip file ./ameri_data/AMF_US-UMd_BASE-BADM_12-5.zip........
Process zip file ./ameri_data/AMF_US-Sne_BASE-BADM_7-5.zip........
AMF_US-Sne_BASE_HH_7-5.csv 2020 exist 17520, need add 48
Process zip file ./ameri_data/AMF_US-Seg_BASE-BADM_20-5.zip........
Process zip file ./ameri_data/AMF_US-NC2_BASE-BADM_10-5.zip........


In [32]:
_row

{'file_name': 'AMF_US-Sne_BASE_HH_7-5.csv',
 'existing_rows': 17520,
 'add_rows': 48}

In [36]:
records

[{'file_name': 'AMF_US-PFs_BASE_HH_1-5.csv',
  'year': '2019',
  'existing_rows': 14592,
  'add_rows': 2928},
 {'file_name': 'AMF_US-ARM_BASE_HH_11-5.csv',
  'year': '2021',
  'existing_rows': 8484,
  'add_rows': 9036},
 {'file_name': 'AMF_US-JRn_BASE_HH_1-5.csv',
  'year': '2019',
  'existing_rows': 4320,
  'add_rows': 13200},
 {'file_name': 'AMF_US-Tw5_BASE_HH_3-5.csv',
  'year': '2020',
  'existing_rows': 1392,
  'add_rows': 16176},
 {'file_name': 'AMF_US-Mi3_BASE_HH_1-5.csv',
  'year': '2019',
  'existing_rows': 17326,
  'add_rows': 194},
 {'file_name': 'AMF_US-PSH_BASE_HH_1-5.csv',
  'year': '2017',
  'existing_rows': 4320,
  'add_rows': 13200},
 {'file_name': 'AMF_US-Rwf_BASE_HH_2-5.csv',
  'year': '2020',
  'existing_rows': 13152,
  'add_rows': 4416},
 {'file_name': 'AMF_US-TrB_BASE_HH_1-5.csv',
  'year': '2020',
  'existing_rows': 14640,
  'add_rows': 2928},
 {'file_name': 'AMF_US-CMW_BASE_HH_2-5.csv',
  'year': '2021',
  'existing_rows': 17519,
  'add_rows': 1},
 {'file_name':

In [39]:
pd_data = pd.DataFrame(records)
pd_data.to_csv('sites_add_rows.csv',index=False)